In [6]:
# librairies
import xml.etree.ElementTree as ET

import time
import requests
from pprint import pprint

import pandas as pd
import csv

from pathlib import Path

In [41]:
# Extraction d'infos sur les auteurs correspondants (reprint='Y')
def extract_CA(file, out_dir, compteur):
    # Lecture XML
    tree = ET.parse(file)
    root = tree.getroot()
    # Créa output
    output = []
    # Traitement
    for rec in root.findall("./REC"):
        # 1 DOI par entrée ?
        dois = rec.findall("dynamic_data/cluster_related/identifiers/identifier")
        if len(dois) != 1:
            #print([x.attrib['value'] for x in dois])
            # clarifier cas où multiples DOIs
            try: 
                assert all(x.attrib['type'] in ("doi", "xref_doi", "parent_book_doi") for x in dois)
            except AssertionError: 
                print(ET.tostring(rec))
                raise
            #assert all(x.attrib['value'] == dois[0].attrib['value'] for x in dois[1:]) #autres dois sont==1er ?
            compteur +=1
            print(compteur, [x.attrib['type'] for x in dois], [x.attrib['value'] for x in dois])
            #raise ValueError("erreur : pas exactement 1 DOI")
        # on récupère le DOI de la publication
        doi = rec.find("dynamic_data/cluster_related/identifiers/identifier").attrib['value']
        # dans l'objet adresses on récupère le nom du CA, le pays de son eta
        addresses = rec.findall("static_data/fullrecord_metadata/addresses/address_name")
        if any(address.findall("names/name[@reprint='Y']") for address in addresses):
            for address in addresses:
                name_eta = address.find("address_spec/full_address").text
                country_eta = address.find("address_spec/country").text
                authors_reprint = address.findall("names/name[@reprint='Y']")
                for author in authors_reprint:
                    name_author = author.find("full_name").text
                    output.append((doi, name_author, country_eta, name_eta))
        else:
            # on laisse infos en "non connu par WOS"
            output.append((doi, "unknown_by_WOS", "unknown_by_WOS", "unknown_by_WOS"))
            continue
        
    # Export output
    df = pd.DataFrame(output, columns = ["doi", "name_author", "country_eta", "name_eta"])
    output_name = file.with_suffix('.csv').name
    df.to_csv(out_dir / output_name, index = False)
    return compteur

In [42]:
p = Path('../data/3.external/WebofScience/notices/')
out_dir = p.with_name('output_python')
out_dir.mkdir(exist_ok=True) #pas d'erreur si existe déjà

compteur = 0
for file in p.glob('*.xml'):
    compteur = extract_CA(file, out_dir, compteur)

1 ['doi', 'xref_doi'] ['10.15252/embr.201847604', '10.15252/embr.201847604']
2 ['doi', 'doi', 'doi', 'doi', 'doi', 'doi', 'doi'] ['10.1371/journal.pbio.3000793', '10.1371/journal.pbio.3000793.r001', '10.1371/journal.pbio.3000793.r002', '10.1371/journal.pbio.3000793.r003', '10.1371/journal.pbio.3000793.r004', '10.1371/journal.pbio.3000793.r005', '10.1371/journal.pbio.3000793.r006']
3 ['doi', 'xref_doi'] ['10.15252/embj.2020106267', '10.15252/embj.2020106267']
4 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-28535-7', '10.1007/978-3-030-28535-7_17']
5 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-17272-5', '10.1007/978-3-030-17272-5_4']
6 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-17272-5', '10.1007/978-3-030-17272-5_1']
7 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-28535-7', '10.1007/978-3-030-28535-7_5']
8 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-13547-8', '10.1007/978-3-030-13547-8_2']
9 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-22942-3', '10.1007/978-3-319-22942-3_

94 ['doi', 'xref_doi'] ['10.1001/jama.2013.107828', '10.1001/jama.2013.107828']
95 ['doi', 'doi', 'doi'] ['10.1016/S0140-6736(18)32782-X', '10.1681/ASN.V1211', '10.1007/s00062-021-01050-0']
96 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-55215-2', '10.1007/978-3-030-55215-2_2']
97 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-64747-6', '10.1007/430_2020_62']
98 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-56700-2', '10.1007/978-3-030-56700-2_12']
99 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-68920-9', '10.1007/12_2020_79']
100 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-42553-1', '10.1007/978-3-030-42553-1_4']
101 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-56700-2', '10.1007/978-3-030-56700-2_8']
102 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-00819-6', '10.1007/978-3-319-00819-6_2']
103 ['parent_book_doi', 'doi'] ['10.1007/ 978-3-030-48457-6', '10.1007/978-3-030-48457-6_3']
104 ['doi', 'doi', 'doi', 'doi', 'doi', 'doi', 'doi'] ['10.1371/journal.pbio.3000843', '10.13

185 ['doi', 'xref_doi'] ['10.1126/scitranslmed.3008538', '10.1126/scitranslmed.3008538']
186 ['parent_book_doi', 'doi'] ['10.1007/978-981-10-6955-0', '10.1007/978-981-10-6955-0_13']
187 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-02771-1', '10.1007/978-3-030-02771-1_5']
188 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-36020-7', '10.1007/978-3-030-36020-7_11']
189 ['parent_book_doi', 'doi'] ['10.1007/978-981-15-1728-0', '10.1007/978-981-15-1728-0_1']
190 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-52640-5', '10.1007/978-3-030-52640-5_15']
191 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-36020-7', '10.1007/978-3-030-36020-7_10']
192 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-52640-5', '10.1007/978-3-030-52640-5_1']
193 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-56700-2', '10.1007/978-3-030-56700-2_10']
194 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-52640-5', '10.1007/978-3-030-52640-5_18']
195 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-36020-7', '10.1007/978-3-0

274 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-02585-8', '10.1007/978-3-319-02585-8_10']
275 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-01270-4', '10.1007/978-3-319-01270-4_11']
276 ['parent_book_doi', 'doi'] ['10.1007/978-3-642-35662-9', '10.1007/978-3-642-35662-9_4']
277 ['doi', 'xref_doi'] ['10.1088/0026-1394/55/1A/04001', '10.1088/1681-7575/aaee2e']
278 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-42312-8', '10.1007/978-3-319-42312-8_3']
279 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-42312-8', '10.1007/978-3-319-42312-8_2']
280 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-17641-3', '10.1007/978-3-319-17641-3_3']
281 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-42360-9', '10.1007/5584_2016_12']
282 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-08428-2', '10.1007/3418_2014_75']
283 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-22138-0', '10.1007/12_2015_317']
284 ['doi', 'xref_doi'] ['10.1097/PAI.0000000000000008', '10.1097/PAI.0000000000000008']
285 ['parent_book

360 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-10298-6', '10.1007/978-3-319-10298-6_2']
361 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-00321-4', '10.1007/978-3-319-00321-4_6']
362 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-40464-2', '10.1007/10_2019_116']
363 ['doi', 'xref_doi'] ['10.1200/JCO.2019.37.15_suppl.4500', '10.1200/JCO.2019.37.15_suppl.4500']
364 ['parent_book_doi', 'doi'] ['10.1515/9781501512001', '10.2138/rmg.2019.85.4']
365 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-42312-8', '10.1007/978-3-319-42312-8_20']
366 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-09477-9', '10.1007/978-3-319-09477-9_12']
367 ['doi', 'xref_doi'] ['10.1080/0144929X.2011.563800', '10.1080/0144929X.2011.563800']
368 ['doi', 'xref_doi'] ['10.1001/jamaneurol.2020.4309', '10.1001/jamaneurol.2020.4309']
369 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-42679-2', '10.1007/978-3-319-42679-2_4']
370 ['parent_book_doi', 'doi'] ['10.1007/978-981-10-2657-7', '10.1007/978-981-10-2657-7_2']
371

...

628 ['doi', 'xref_doi'] ['10.1016/j.cub.2018.07.056', '10.1016/j.cub.2018.07.056']
629 ['doi', 'doi'] ['10.1098/RSPA.2016.0117', '10.1016/j.dsr.2018.01.001']
630 ['doi', 'doi'] ['10.1108/MD-03-2018-0324', '10.1108/EMJB-10-2021-0154']
631 ['parent_book_doi', 'doi'] ['10.1007/978-1-4939-2895-8', '10.1007/978-1-4939-2895-8_2']
632 ['doi', 'xref_doi'] ['10.1016/j.meegid.2011.01.004', '10.1016/j.meegid.2011.01.004']
633 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-00828-8', '10.1007/978-3-319-00828-8_7']
634 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-02585-8', '10.1007/978-3-319-02585-8_5']
635 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-01270-4', '10.1007/978-3-319-01270-4_9']
636 ['parent_book_doi', 'doi'] ['10.1007/978-3-642-40746-8', '10.1007/12_2012_201']
637 ['doi', 'xref_doi'] ['10.1089/ham.2014.1082', '10.1089/ham.2014.1082']
638 ['doi', 'xref_doi'] ['10.1016/j.therap.2015.11.003', '10.1016/j.therap.2015.11.002']
639 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-25474-6', 

716 ['parent_book_doi', 'doi'] ['10.1007/978-981-10-2657-7', '10.1007/978-981-10-2657-7_13']
717 ['doi', 'xref_doi'] ['10.1038/523526a', '10.1038/523526a']
718 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-17641-3', '10.1007/978-3-319-17641-3_16']
719 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-17641-3', '10.1007/978-3-319-17641-3_4']
720 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-11970-0', '10.1007/978-3-319-11970-0_9']
721 ['parent_book_doi', 'doi'] ['10.1007/978-1-4939-3509-3', '10.1007/978-1-4939-3509-3_3']
722 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-24630-7', '10.1007/3418_2015_137']
723 ['parent_book_doi', 'doi'] ['10.1515/9781501512001', '10.2138/rmg.2019.85.16']
724 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-09477-9', '10.1007/978-3-319-09477-9_20']
725 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-39406-0', '10.1007/978-3-319-39406-0_5']
726 ['doi', 'xref_doi'] ['10.1126/science.aaf6016', '10.1126/science.aaf6016']
727 ['doi', 'xref_doi'] ['10.1097/DBP.0000

809 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-19396-0', '10.1007/3418_2015_101']
810 ['parent_book_doi', 'doi'] ['10.1007/978-3-642-55068-3', '10.1007/128_2014_530']
811 ['doi', 'xref_doi'] ['10.1080/09500839.2012.757662', '10.1080/09500839.2012.757662']
812 ['doi', 'doi', 'doi', 'doi', 'doi', 'doi', 'doi'] ['10.1371/journal.pcbi.1007250', '10.1371/journal.pcbi.1007250.r001', '10.1371/journal.pcbi.1007250.r002', '10.1371/journal.pcbi.1007250.r003', '10.1371/journal.pcbi.1007250.r004', '10.1371/journal.pcbi.1007250.r005', '10.1371/journal.pcbi.1007250.r006']
813 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-44465-9', '10.1007/978-3-319-44465-9_8']
814 ['doi', 'xref_doi'] ['10.17660/ActaHortic.2017.1160.3', '10.17660/ActaHortic.2017.1160.3']
815 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-49928-0', '10.1007/82_2020_207']
816 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-49928-0', '10.1007/82_2020_209']
817 ['doi', 'xref_doi'] ['10.1525/msem.2017.33.2.220', '10.1525/msem.2017.33

898 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-22081-9', '10.1007/128_2015_624']
899 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-59837-2', '10.1007/978-3-030-59837-2_2']
900 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-59837-2', '10.1007/978-3-030-59837-2_1']
901 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-68920-9', '10.1007/12_2020_76']
902 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-56700-2', '10.1007/978-3-030-56700-2_2']
903 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-52463-0', '10.1007/978-3-030-52463-0_2']
904 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-61807-0', '10.1007/978-3-030-61807-0_10']
905 ['doi', 'xref_doi'] ['10.1515/FORM.2011.105', '10.1515/FORM.2011.105']
906 ['parent_book_doi', 'doi'] ['10.1007/978-3-642-45059-4', '10.1007/128_2013_431']
907 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-18585-9', '10.1007/978-3-319-18585-9_14']
908 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-67144-4', '10.1007/978-3-319-67144-4_4']
909 ['parent_book_doi', 'd

992 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-46886-6', '10.1007/978-3-030-46886-6_5']
993 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-46886-6', '10.1007/978-3-030-46886-6_3']
994 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-46886-6', '10.1007/978-3-030-46886-6_6']
995 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-26133-1', '10.1007/82_2015_482']
996 ['doi', 'xref_doi'] ['10.4193/Rhin19.080', '10.4193/Rhin17.253']
997 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-26009-5', '10.1007/3418_2018_6']
998 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-74908-2', '10.1007/978-3-319-74908-2_2']
999 ['parent_book_doi', 'doi'] ['10.1007/978-3-030-26009-5', '10.1007/3418_2018_10']
1000 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-74908-2', '10.1007/978-3-319-74908-2_6']
1001 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-92420-5', '10.1007/978-3-319-92420-5_8']
1002 ['parent_book_doi', 'doi'] ['10.1007/978-3-319-95186-7', '10.1007/978-3-319-95186-7_2']
1003 ['parent_book_doi', 'doi']